This Jupyter notebook complements the paper “Near-optimal Circuit construction via Cartan decomposition” found at [link to arXiv here]. Here, an explicit decomposition of an  $SU(4)$ operator is implemented, according to the steps and methods discussed in detail in the paper. The goal of this code is to show explicitly, in terms of the matrix representation of the operators, how the internal parameters of such a decomposition can be determined. This code is far from optimal in terms of implementation and run-time and serves simply as a pedagogical and applied example of how the Cartan decomposition for unitary operators described on the paper works in practice.

# Explicit Cartan decomposition of $SU(4)$ operator

In [2]:
import pennylane as qml
import numpy as np
from sympy import *
from sympy import symbols

In [3]:
dev1 = qml.device('default.qubit', wires=2)

## Definition of symbolic representations of operators for quantum computations

Here the symbolic variables are defined. These are the variables appearing in the $M_i$ and $N_i$ matrices of the decomposition (see equations (7) and (8) in the paper) and as the variables of the the remaining SU(2) rotations. Note that most QC software does not allow the construction and manipulation of symbolic operators, so from now on we will work with the matrix representation of the respective quantum operators.

In [4]:
a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z =symbols("a b c d e f g h i j k l m n o p q r s t u v w x y z")

## Symbolic SU(2) Rotations around x,y,z axes.

def ROTX(x):
    return Matrix([[cos(x/2), -I*sin(x/2)], [-I*sin(x/2), cos(x/2)]])
def ROTY(y):
    return Matrix([[cos(y/2), -sin(y/2)], [sin(y/2), cos(y/2)]])
def ROTZ(z):
    return Matrix([[cos(z/2)- I*sin(z/2), 0], [0, cos(z/2)+ I*sin(z/2)]])


## CNOT(0,1) AND CNOT(1,0) respectively

CNOT= Matrix([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
CNOT2= Matrix([[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]])

### Comparisson with Pennylane circuit and matrix representation

For consistency, we want to verify that the circuit we construct with the symbolic matrix-operators matches the outcome of the equivalent quantum circuit implemented using Pennylane software. In this case, we create a 2-qubit circuit with an RX rotation by pi/4 followed by a pi/8 rotation around the y-axis, then a pi/4 rotation around the z-axis and finally a CNOT gate. The circuit is shown below:

In [5]:
@qml.qnode(dev1)
def circuit(params):
    qml.RX(params[0], wires=1)
    qml.RY(params[1], wires=1)
    qml.RZ(params[2], wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1))
params = [np.pi/4,np.pi/8,np.pi/4]
print(qml.draw(circuit)(params))

0: ───────────────────────────────╭●─┤     
1: ──RX(0.79)──RY(0.39)──RZ(0.79)─╰X─┤  <Z>


The pennylane matrix representation of the above circuit is as follows (note the reversed order of operations with respect to the circuit above as qubits come from the left on circuits but matrix representations act from the right on qubits):

In [6]:
op = qml.prod(qml.CNOT(wires=[0,1]),qml.RZ(np.pi/4,wires=1),qml.RY(np.pi/8, wires=1),qml.RX(np.pi/4, wires=1))
Matrix(qml.matrix(op))

Matrix([
[ 0.86572291771948 - 0.277785116509801*I, -0.310152684699878 - 0.277785116509801*I,                                       0,                                        0],
[0.310152684699878 - 0.277785116509801*I,   0.86572291771948 + 0.277785116509801*I,                                       0,                                        0],
[                                      0,                                        0, 0.310152684699878 - 0.277785116509801*I,   0.86572291771948 + 0.277785116509801*I],
[                                      0,                                        0,  0.86572291771948 - 0.277785116509801*I, -0.310152684699878 - 0.277785116509801*I]])

We now want to compare the above matrix with the matrix-representation we obtain using the symbolic matrices defined above:

In [24]:
simplify(simplify(simplify(CNOT@np.kron(eye(2),ROTZ(np.pi/4)@ROTY(np.pi/8)@ROTX(np.pi/4)))))

Matrix([
[ 0.86572291771948 - 0.277785116509801*I, -0.310152684699878 - 0.277785116509801*I,                                       0,                                        0],
[0.310152684699878 - 0.277785116509801*I,   0.86572291771948 + 0.277785116509801*I,                                       0,                                        0],
[                                      0,                                        0, 0.310152684699878 - 0.277785116509801*I,   0.86572291771948 + 0.277785116509801*I],
[                                      0,                                        0,  0.86572291771948 - 0.277785116509801*I, -0.310152684699878 - 0.277785116509801*I]])

 As expected we obtain the same matrix representation of the quantum circuit.

## Explicit SU(4) Cartan decomposition

Starting with an arbitrary $SU(4)$ operator $V$, after carrying the Cartan decomposition $V= K_1 exp(y) K_2$ (as described in equation (2) on the paper) and after block-diagonalizing the exponential term $exp(y)=M(a,b,c)$ as described in section III of the paper, we obtain the following circuit:

![SNOWFALL](circuit.jpg)


In order to explicitly compute the parameters $a,b,c ...$ we need to first compute the matrix representation of the above circuit.


In [25]:
## Matrix representation of the circuit diagonalizing and executing the exponential term M(a,b,c)

A=CNOT @np.kron(eye(2),ROTZ(-2*c))@CNOT@CNOT2@ np.kron(eye(2),ROTZ(-np.pi/2))@CNOT@np.kron(eye(2),ROTY(-2*b))@CNOT@np.kron(eye(2),ROTY(2*a))@np.kron(eye(2),ROTZ(np.pi/2))@CNOT2
A=simplify(A)

## SU(2) rotations on the left-hand-side of the circuit
B=np.kron(eye(2),ROTZ(d)@ROTY(e)@ROTX(f))@ np.kron(ROTZ(g)@ROTY(h)@ROTX(i),eye(2))
B=simplify(B)
           
## SU(2) rotations on the right-hand-side of the circuit
C=np.kron(eye(2),ROTZ(j)@ROTY(k)@ROTX(l))@np.kron(ROTZ(m)@ROTY(n)@ROTX(r),eye(2))
C=simplify(C)


Finally, to obtain the matrix representation of the circuit above we simply multiply the matrices together

In [21]:
U=C@A@B

## Computing the decomposition parameters

We now have the matrix representation of the Cartan decomposition of a $SU(4)$ operator $V$. In order to compute the parameters $a,b,c ...$, we need to compare each entry of the $U$ matrix we obtained above with the desired matrix $V$ we want to decompose. Note that we have $4x4$ matrices and only $15$ parameters, however, we have the additional constraint that $U \in SU(4)$, which removes one degree of freedom. This leaves us with a system of 15 unknown variables and 15 non-linear equations.

The system of equations is not a trivial one. For reference, let us see how one of the entries of the matrix $U$ we need to solve for looks like:

In [23]:
U[0,0]

(sin(e/2)*cos(f/2) - I*sin(f/2)*cos(e/2))*(sin(h/2)*cos(i/2) - I*sin(i/2)*cos(h/2))*(1.0*(sin(k/2)*cos(l/2) + I*sin(l/2)*cos(k/2))*(sin(n/2)*cos(r/2) + I*sin(r/2)*cos(n/2))*exp(I*c)*exp(-I*j/2 - I*m/2)*cos(a - b) - (0.707106781186548 + 0.707106781186547*I)**2*(I*sin(k/2)*sin(l/2) + cos(k/2)*cos(l/2))*(I*sin(n/2)*sin(r/2) + cos(n/2)*cos(r/2))*exp(I*c)*exp(-I*j/2 - I*m/2)*sin(a - b))*exp(I*d/2 + I*g/2) + (sin(e/2)*cos(f/2) - I*sin(f/2)*cos(e/2))*(I*sin(h/2)*sin(i/2) + cos(h/2)*cos(i/2))*(-(0.707106781186548 + 0.707106781186547*I)**2*(I*sin(c) - cos(c))*(sin(n/2)*cos(r/2) + I*sin(r/2)*cos(n/2))*(I*sin(k/2)*sin(l/2) + cos(k/2)*cos(l/2))*exp(-I*j/2 - I*m/2)*sin(a + b) - 1.0*(sin(k/2)*cos(l/2) + I*sin(l/2)*cos(k/2))*(I*sin(n/2)*sin(r/2) + cos(n/2)*cos(r/2))*exp(-I*c)*exp(-I*j/2 - I*m/2)*cos(a + b))*exp(I*d/2 - I*g/2) + (sin(h/2)*cos(i/2) - I*sin(i/2)*cos(h/2))*(I*sin(e/2)*sin(f/2) + cos(e/2)*cos(f/2))*((0.707106781186548 - 0.707106781186547*I)**2*(I*sin(c) - cos(c))*(sin(k/2)*cos(l/2) + I*si

E.g. If we would like to decompose the $4x4$ unitary matrix using the method described above, we would need to solve the following system of equations:

In [ ]:
system = [eye(4)-U, eye(4)- U@U]
vars = [a,b,c,d,e,f,g,h,i,j,k,l,m,n,r]
eq= eye(4)-U
solution = solve(eq, vars)


Due to the complexity of the system of equations exact solvers are extremely inefficient and often run into runtime errors. Thus, numerical solvers should be used. However, this lies outside the scope of our work and area of knowledge.